In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# !pip install unrar
# !unrar x /content/drive/MyDrive/Images.rar

In [ ]:
import cv2, os
import matplotlib.pyplot as plt
import itertools, numpy as np
from tensorflow.keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
def preprocessImage(path):
    my_image = load_img(path, target_size=(71, 71))

    my_image = img_to_array(my_image)
    # my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
    my_image = preprocess_input(my_image)
    return my_image

In [ ]:
realImages=[]
fakeImages=[]
for i in np.random.choice(os.listdir('/content/drive/MyDrive/Siamese Images/Real'), 100, replace=False):
    img = preprocessImage(f'/content/drive/MyDrive/Siamese Images/Real/{i}')
    realImages.append([img,1])
for i in np.random.choice(os.listdir('/content/drive/MyDrive/Siamese Images/Fake'), 100, replace=False):
    img = preprocessImage(f'/content/drive/MyDrive/Siamese Images/Fake/{i}')
    fakeImages.append([img,0])

In [ ]:
Images = realImages + fakeImages

In [ ]:
imagePairs = []
labels = []
for i in itertools.combinations(Images,2):
    if i[0][1] == i[1][1]:
        imagePairs.append([i[0][0], i[1][0]])
        labels.append(1)
    else:
        imagePairs.append([i[0][0], i[1][0]])
        labels.append(0)
labels = np.array(labels, dtype=np.float32)
imagePairs = np.array(imagePairs)

In [ ]:
import numpy as np
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K


In [ ]:
def create_siamese_xception(input_shape):
    base_xception = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

    # Define two input tensors for the siamese network
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # Use the same Xception model for both inputs
    output_a = base_xception(input_a)
    output_b = base_xception(input_b)

    # Flatten the outputs and compute Euclidean distance
    output_a = Flatten()(output_a)
    output_b = Flatten()(output_b)

    # Calculate Euclidean distance between the outputs of the base network
    distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([output_a, output_b])

    # Output layer
    output = Dense(1, activation='sigmoid')(distance)

    # Create the Siamese model
    siamese_model = Model(inputs=[input_a, input_b], outputs=output)
    return siamese_model


In [ ]:
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

    return contrastive_loss


In [ ]:
# Define input shape (adjust according to your dataset)
input_shape = (71, 71, 3)

# Create Siamese model
siamese_model = create_siamese_xception(input_shape)

# Compile the model with contrastive loss and Adam optimizer
siamese_model.compile(loss=contrastive_loss_with_margin(margin=1.0), optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Train the model with your data
siamese_model.fit([imagePairs[:,0,:,:,:], imagePairs[:,1,:,:,:]], labels, epochs=20, batch_size=32)


83683744/83683744 [==============================] - 3s 0us/step
Epoch 1/20
622/622 [==============================] - 127s 143ms/step - loss: 0.1832 - accuracy: 0.3397
Epoch 2/20
622/622 [==============================] - 89s 144ms/step - loss: 0.0052 - accuracy: 0.0054
Epoch 3/20
622/622 [==============================] - 89s 143ms/step - loss: 0.0030 - accuracy: 0.0033
Epoch 4/20
622/622 [==============================] - 89s 143ms/step - loss: 0.0025 - accuracy: 0.0032
Epoch 5/20
622/622 [==============================] - 89s 144ms/step - loss: 5.0944e-04 - accuracy: 5.5276e-04
Epoch 6/20
622/622 [==============================] - 89s 143ms/step - loss: 3.8668e-05 - accuracy: 5.0251e-05
Epoch 7/20
622/622 [==============================] - 89s 143ms/step - loss: 2.7999e-06 - accuracy: 0.0000e+00
Epoch 8/20
622/622 [==============================] - 89s 143ms/step - loss: 5.7592e-06 - accuracy: 0.0000e+00
Epoch 9/20
622/622 [==============================] - 89s 143ms/step - loss: 3

In [ ]:
siamese_model.save('/content/siamese_model.h5')


In [ ]:
xception = siamese_model.layers[2]

In [ ]:
xception.save('/content/xceptionmodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
